In [10]:
import numpy as np
import pandas as pd
import math
import sys, os
#import seaborn as sns
#from scipy import stats
#from pathlib import Path
#import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import fetch_california_housing
#import math
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
import optuna
from optuna.samplers import TPESampler
#import lightgbm as lgbm
#from xgboost import XGBRegressor
import xgboost as xgb
#from catboost import CatBoostRegressor
#from lightgbm.sklearn import LGBMRegressor
#from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
pd.set_option('display.max_columns', 500)

In [2]:
train_time = 1 * 60 * 60
objective = 'xgbregressor'

In [3]:
sys.path.append(os.path.realpath('..'))

In [43]:
train_df = pd.read_csv(r"..\data\train.csv", index_col=0)
test_df = pd.read_csv(r"..\data\test.csv", index_col=0)
sample_sub = pd.read_csv(r"..\data\sample_submission.csv", index_col=0)

In [44]:
total = train_df.isnull().sum().sort_values(ascending=False)
percent = (train_df.isnull().sum()/train_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

,Total,Percent
Age,0,0.0
StockOptionLevel,0,0.0
Over18,0,0.0
OverTime,0,0.0
PercentSalaryHike,0,0.0
PerformanceRating,0,0.0
RelationshipSatisfaction,0,0.0
StandardHours,0,0.0
TotalWorkingYears,0,0.0
BusinessTravel,0,0.0


In [45]:
del train_df['Over18']
del train_df['EmployeeCount']
del train_df['StandardHours']

In [46]:
columns_to_vectorize = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
for vector_target in columns_to_vectorize:
    print(vector_target)
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

BusinessTravel
Department
EducationField
Gender
JobRole
MaritalStatus
OverTime


In [47]:
features = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction','HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
        'BusinessTravel_v', 'Department_v', 'EducationField_v','Gender_v', 'JobRole_v', 'MaritalStatus_v', 'OverTime_v']
target = ['Attrition']

In [48]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [51]:
y_min = Y.min()
y_max = Y.max()

print(y_min, y_max)

def my_rmse(y_true, y_hat):
    y_true[y_true < y_min] = y_min
    y_true[y_true > y_max] = y_max
    
    y_hat[y_hat < y_min] = y_min
    y_hat[y_hat > y_max] = y_max
    
    y_true_nan = np.isnan(y_true)
    y_hat_nan = np.isnan(y_hat)
    
    if y_true_nan.sum() > 0:
        print(y_true_nan.sum())
        np.where(y_true_nan, np.ma.array(y_true, mask=np.isnan(y_true)).mean(axis=0), y_true)
    if y_hat_nan.sum() > 0:
        print(y_hat_nan.sum())
        np.where(y_hat_nan, np.ma.array(y_hat, mask=np.isnan(y_hat)).mean(axis=0), y_hat)
    
    return mean_squared_error(y_true, y_hat, squared=False)

0.0 1.0


In [52]:
RANDOM_STATE = 12 
FOLDS = 5
param_grid_history = {}

def render_model(param_grid):
    reg = xgb.XGBModel(
        # These parameters should help with trial speed.
        objective='binary:logistic',
        tree_method='gpu_hist',
        booster='gbtree',
        predictor='gpu_predictor',
        n_jobs=4,
        eval_metric='auc',
        early_stopping_rounds=100,
        **param_grid
    )
    return reg
    
def train_model(reg, X_train, y_train, X_valid, y_valid):
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)

    preds = reg.predict(train_df[features])
    fpr, tpr, _ = roc_curve(train_df[target], preds)
    roc_auc = auc(fpr, tpr)
    return reg, roc_auc

def objective_v2(trial):
    scores = []
    param_grid = {
        'max_depth': trial.suggest_int('max_depth', 2, 20),
        'n_estimators': trial.suggest_int('n_estimators', 2, 1000, 1), 
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.9),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.001, 0.9),
        #'eta': trial.suggest_float('eta', 0.07, 0.113), 
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 40), 
    } 
        
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        print(10*"=", f"Fold={fold+1}", 10*"=")
        X_train, X_valid = train_df.iloc[train_idx][features], train_df.iloc[valid_idx][features]
        y_train , y_valid = train_df[target].iloc[train_idx] , train_df[target].iloc[valid_idx] 
        
        reg = render_model(param_grid)
        reg, roc_auc = train_model(reg, X_train, y_train, X_valid, y_valid)

        scores.append(roc_auc)
    mean_scores = np.mean(scores)
    param_grid_history[mean_scores] = param_grid

    return mean_scores

In [53]:
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name=objective)
study.optimize(objective_v2, timeout=train_time)

[I 2023-01-17 22:32:55,112] A new study created in memory with name: xgbregressor


========== Fold=1 ==========
========== Fold=2 ==========
========== Fold=3 ==========
========== Fold=4 ==========
========== Fold=5 ==========


[I 2023-01-17 22:32:59,990] Trial 0 finished with value: 0.9737102234258632 and parameters: {'max_depth': 15, 'n_estimators': 295, 'learning_rate': 0.8406310611213746, 'min_child_weight': 0.07213041944690822, 'subsample': 0.5662750590000322, 'colsample_bytree': 0.3291835320770484, 'reg_lambda': 10.252531206651918}. Best is trial 0 with value: 0.9737102234258632.


========== Fold=1 ==========
========== Fold=2 ==========
========== Fold=3 ==========
========== Fold=4 ==========
========== Fold=5 ==========


[I 2023-01-17 22:33:02,690] Trial 1 finished with value: 0.9325930941096818 and parameters: {'max_depth': 5, 'n_estimators': 809, 'learning_rate': 0.3418709884216741, 'min_child_weight': 0.44648216741886215, 'subsample': 0.30717475510759196, 'colsample_bytree': 0.9731879834597674, 'reg_lambda': 28.7388200634674}. Best is trial 0 with value: 0.9737102234258632.


...

========== Fold=1 ==========
========== Fold=2 ==========
========== Fold=3 ==========
========== Fold=4 ==========
========== Fold=5 ==========


[I 2023-01-17 22:33:11,387] Trial 3 finished with value: 0.9743067027758971 and parameters: {'max_depth': 9, 'n_estimators': 253, 'learning_rate': 0.28538262872574705, 'min_child_weight': 0.23068587358231543, 'subsample': 0.8548059100574196, 'colsample_bytree': 0.5699286281948834, 'reg_lambda': 6.016423739207473}. Best is trial 3 with value: 0.9743067027758971.


========== Fold=1 ==========
========== Fold=2 ==========
========== Fold=3 ==========
========== Fold=4 ==========
========== Fold=5 ==========


[W 2023-01-17 22:33:15,047] Trial 4 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "D:\source\repos\venv\Python310\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_38916\2563366594.py", line 47, in objective_v2
    reg, roc_auc = train_model(reg, X_train, y_train, X_valid, y_valid)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_38916\2563366594.py", line 20, in train_model
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
  File "D:\source\repos\venv\Python310\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "D:\source\repos\venv\Python310\lib\site-packages\xgboost\sklearn.py", line 1025, in fit
    self._Booster = train(
  File "D:\source\repos\venv\Python310\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "D:\sour

KeyboardInterrupt: 

In [ ]:
parm_grid_bytes = pickle.dumps(param_grid_history)
with open(f"rendered_data/{objective}_bytes.hex", "wb") as binary_file:
    binary_file.write(parm_grid_bytes)

In [57]:
def train(param_grid):
    reg = render_model(param_grid)
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df[features], train_df[target])):
        print(10*"=", f"FINAL TRAINING Fold={fold+1}", 10*"=")
        X_train, X_valid = train_df.iloc[train_idx][features], train_df.iloc[valid_idx][features]
        y_train , y_valid = train_df[target].iloc[train_idx] , train_df[target].iloc[valid_idx] 
        reg, roc_auc = train_model(reg, X_train, y_train, X_valid, y_valid)
    return reg

percent = math.ceil(len(param_grid_history.keys()) * .1)
top = sorted(list(param_grid_history.keys()))[-percent:]

train_preds = []
test_preds = []
for key in tqdm(top):
    model = train(param_grid_history[key])
    train_preds.append(model.predict(train_df[features]))
    test_preds.append(model.predict(test_df[features]))

train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

========== FINAL TRAINING Fold=1 ==========
========== FINAL TRAINING Fold=2 ==========
========== FINAL TRAINING Fold=3 ==========
========== FINAL TRAINING Fold=4 ==========
========== FINAL TRAINING Fold=5 ==========


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.85s/it]


In [58]:
train_submission = pd.DataFrame(data={'id': train_df.index, target[0]: train_final_preds})
train_submission.to_csv(fr'rendered_data/{objective}_train_submission.csv', index=False)

test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: test_final_preds})
test_submission.to_csv(fr'rendered_data/{objective}_test_submission.csv', index=False)